In [13]:
pip install flask scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, request, jsonify
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from joblib import dump, load
import os

# Directory to store trained models
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)


In [2]:
app = Flask(__name__)

# Available models
MODEL_CLASSES = {
    "RandomForest": RandomForestClassifier,
    "GradientBoosting": GradientBoostingClassifier
}


In [4]:
@app.route('/models', methods=['GET'])
def list_models():
    """Return a list of available model classes for training."""
    return jsonify(list(MODEL_CLASSES.keys()))


In [5]:
@app.route('/train', methods=['POST'])
def train_model():
    """Train a model with specified hyperparameters."""
    data = request.get_json()
    model_name = data.get('model_name')
    params = data.get('params', {})

    if model_name not in MODEL_CLASSES:
        return jsonify({"error": "Model not available"}), 400

    X, y = make_classification(n_samples=1000, n_features=20)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_class = MODEL_CLASSES[model_name]
    model = model_class()

    grid_search = GridSearchCV(model, params, cv=3)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    model_path = f"{MODEL_DIR}/{model_name}.joblib"
    dump(best_model, model_path)

    return jsonify({"message": f"{model_name} trained successfully", "best_params": grid_search.best_params_})


In [6]:
@app.route('/predict', methods=['POST'])
def predict():
    """Predict using a specified trained model."""
    data = request.get_json()
    model_name = data.get('model_name')
    input_data = data.get('input_data')

    model_path = f"{MODEL_DIR}/{model_name}.joblib"
    if not os.path.exists(model_path):
        return jsonify({"error": "Model not found"}), 404

    model = load(model_path)
    prediction = model.predict([input_data])

    return jsonify({"prediction": prediction.tolist()})


In [7]:
@app.route('/delete_model', methods=['DELETE'])
def delete_model():
    """Delete a specified trained model."""
    data = request.get_json()
    model_name = data.get('model_name')
    
    model_path = f"{MODEL_DIR}/{model_name}.joblib"
    if os.path.exists(model_path):
        os.remove(model_path)
        return jsonify({"message": f"{model_name} deleted successfully"})
    else:
        return jsonify({"error": "Model not found"}), 404


In [8]:
@app.route('/retrain', methods=['POST'])
def retrain_model():
    """Re-train an existing model with new hyperparameters."""
    return train_model()


In [9]:
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "API is running"})


In [10]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\JUAN\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
